In [ ]:
!wget https://github.com/DataTalksClub/machine-learning-zoomcamp/releases/download/dl-models/clothing-model-new.keras

In [1]:
# !pip install git+https://github.com/onnx/tensorflow-onnx.git


  Cloning https://github.com/onnx/tensorflow-onnx.git to /tmp/pip-req-build-ot8sx8lm
  Running command git clone --filter=blob:none --quiet https://github.com/onnx/tensorflow-onnx.git /tmp/pip-req-build-ot8sx8lm
  Resolved https://github.com/onnx/tensorflow-onnx.git to commit 4fed7de9534b6a084f7f2326bae775545bd97f9e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 15.2 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tf2onnx: filename=tf2onnx-1.16.1-py3-none-any.whl size=456807 sha256=d90be803bab9561bf4246a3598ac30bcb355becbfda4693b4

In [2]:
from tensorflow import keras

model = keras.models.load_model('keras/models/clothing-model-new.keras')
model.export("clothing-model-new_savedmodel")

2025-12-13 14:12:46.137830: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1765617167.031043   21892 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21768 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


INFO:tensorflow:Assets written to: clothing-model-new_savedmodel/assets


INFO:tensorflow:Assets written to: clothing-model-new_savedmodel/assets


Saved artifact at 'clothing-model-new_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): List[TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name='input_layer_6')]
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  128013483645200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128013483645776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128013483645584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128013483646160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128013483645008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128013483646544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128013483645392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128013483646352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128013483645968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128013483647504: TensorSpec(shape=(), dtype=tf.resource, name=N

In [ ]:
# python -m tf2onnx.convert --saved-model clothing-model-new_savedmodel --opset 13 --output clothing-model-new.onnx

In [18]:
import onnxruntime as ort
# import onnxruntime-gpu as ort

In [20]:

onnx_model_path = "clothing-model-new.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CUDAExecutionProvider"])

In [30]:

inputs = session.get_inputs()
outputs = session.get_outputs()

In [21]:

input_name = inputs[0].name
output_name = outputs[0].name

In [31]:
input_name

'input_layer_6'

In [32]:
output_name

'output_0'

In [50]:
from keras_image_helper import create_preprocessor

preprocessor = create_preprocessor('xception', target_size=(299, 299))

url = 'http://bit.ly/mlbookcamp-pants'
url = 'https://cdn11.bigcommerce.com/s-b1nxqcmq/images/stencil/760x760/products/550/4152/Oxford_Button_Down_Shirt_Lt._Blue__27710.1762839125.png?c=2'
X = preprocessor.from_url(url)

In [51]:
result = session.run([output_name], {input_name: X})
predictions = result[0][0].tolist()

classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

d = dict(zip(classes, predictions))
sorted(d.items(), key=lambda kv: kv[1], reverse=True)

[('longsleeve', 4.0070343017578125),
 ('shirt', 3.8237345218658447),
 ('outwear', 3.4062464237213135),
 ('t-shirt', -1.2262977361679077),
 ('dress', -1.8901089429855347),
 ('pants', -2.2954206466674805),
 ('hat', -3.2785098552703857),
 ('skirt', -3.716797351837158),
 ('shorts', -3.828216552734375),
 ('shoes', -3.838379383087158)]

In [7]:
import onnxruntime as ort
from keras_image_helper import create_preprocessor

preprocessor = create_preprocessor("xception", target_size=(299, 299))

session = ort.InferenceSession(
    "clothing-model-new.onnx", providers=["CPUExecutionProvider"]
)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

classes = [
    "dress",
    "hat",
    "longsleeve",
    "outwear",
    "pants",
    "shirt",
    "shoes",
    "shorts",
    "skirt",
    "t-shirt",
]


def predict(url):
    X = preprocessor.from_url(url)
    result = session.run([output_name], {input_name: X})
    float_predictions = result[0][0].tolist()
    return dict(zip(classes, float_predictions))


def lambda_handler(event, context):
    url = event["url"]
    result = predict(url)
    return result